## Wedge Project - Task Two



In [9]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import os
import polars as pl
import pandas as pd
import random

client = bigquery.Client(project="wedge-project-jbangtson")
data_directory = "E:\\College\\Fall 2024\\ADA\\Wedge\\Wedge_Project\\data\\unzipped\\"


In [7]:
wedge_task_two = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data.txt', sep='\t')

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\380248305.py:1: SyntaxWarning: invalid escape sequence '\C'
  wedge_task_two = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data.txt', sep='\t')
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\380248305.py:1: DtypeWarning: Columns (33,34,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  wedge_task_two = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data.txt', sep='\t')


Run the code above to view all transactions for card owners: 

48289, 48420, 56191, 20300, 48996, 56191, and 16551.

---

In [ ]:


# List of columns
columns = ['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id']

# Create an empty Polars DataFrame with the specified columns
gbq_query_df = pl.DataFrame({col: [] for col in columns})

print(gbq_query_df)

In [3]:
# Function to run a query
def run_query(query):
    try:
        query_job = client.query(query)
        results = query_job.result()

        bytes_processed = query_job.total_bytes_processed
        mb_processed = bytes_processed / (1024 ** 2)
        cost_per_tb = 5.0


        tb_processed = bytes_processed / (1024 ** 4)  # Convert bytes to terabytes
        estimated_cost = tb_processed * cost_per_tb

        # Display the processed data and estimated cost
        print(f"Data processed: {mb_processed:.2f} MB")
        

        print(f"Estimated bytes processed: {bytes_processed}")
        print(f"Estimated cost: ${estimated_cost:.10f}\n\n")

        print(f"Estimated bytes processed against a full year of data: {bytes_processed*50}")
        print(f"Estimated cost against a full year of data: ${estimated_cost*50:.20f}")
        print(f"Estimated cost against a full year of data every 6 hours: ${(estimated_cost*50)*(4*365):.20f}\n\n---------------")

        

        return results
    except GoogleAPIError as e:
        print(f"Error running query: {e}")
        return None

***This next function is a modified version of the function above

In [3]:
# Function to run a query
def run_query2(query):
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        #results = query_job.result()

        bytes_processed = query_job.total_bytes_processed
        mb_processed = bytes_processed / (1024 ** 2)
        cost_per_tb = 5.0


        tb_processed = bytes_processed / (1024 ** 4)  # Convert bytes to terabytes
        estimated_cost = tb_processed * cost_per_tb

        # Display the processed data and estimated cost
        print(f"Data processed: {mb_processed:.2f} MB")
        

        print(f"Estimated bytes processed: {bytes_processed}")
        print(f"Estimated cost: ${estimated_cost:.10f}\n\n")

        print(f"Estimated bytes processed against a full year of data: {bytes_processed*50}")
        print(f"Estimated cost against a full year of data: ${estimated_cost*50:.20f}")
        print(f"Estimated cost against a full year of data every 6 hours: ${(estimated_cost*50)*(4*365):.20f}\n\n---------------")

        

        return df.astype(str)
    except GoogleAPIError as e:
        print(f"Error running query: {e}")
        return None

In [ ]:
# all_card_nums = []

# distinct_card_nums = """
# SELECT
#   DISTINCT(card_no) as card_no
# FROM
#   `the_wedge_dataset.transArchive_*`
# WHERE card_no != 3
# """

# # Run the query and display results
# results = run_query(distinct_card_nums)
# if results:
#     for row in results:
#         print(f"{row.card_no}")
#         all_card_nums.append(row.card_no)


    





In [ ]:


# Select 20 random numbers from the all_card_nums list
random_card_nums = random.sample(all_card_nums, 20)
print(random_card_nums)

# Write the random_card_nums list to a tab-delimited text file
with open('random_card_nums.txt', 'w') as f:
    for num in random_card_nums:
        f.write(f"{num}\t")

## GBQ Owner Queries

This code defines a function save_owner_query_to_file() that retrieves and saves data from a Google BigQuery dataset to a CSV file. 

The function takes two arguments: yearOfQuery (the year to filter the data by) and table-by-table 
(a boolean flag that controls whether to process each table individually or as a group). 


In [1]:


def save_owner_query_to_file(yearOfQuery, tableByTable=False):
    
  if tableByTable:

    for idx, file in enumerate(os.listdir(data_directory)):
      owner_query_df = pd.DataFrame()
      if f"transArchive_{yearOfQuery}" in file:
      

        owner_gbq_query = f"""
        SELECT
          *,
        SAFE_CAST(Scale AS INT64) AS IntScale
        FROM
          `the_wedge_dataset.transArchive_{yearOfQuery}*`
        WHERE card_no = 48289.0 OR card_no = 48420.0 OR card_no = 56191.0 OR card_no = 20300.0 OR card_no = 48996.0 OR card_no = 56191.0 OR card_no = 16551.0 
        """

        temp_df = run_query2(owner_gbq_query)

        # #temp_df = pl.DataFrame(results)

        # rows = [dict(row) for row in results]
        # #columns = list(rows[0].keys()) if rows else []

        # # Create Polars DataFrame from the rows
        # temp_df = pl.DataFrame(rows)

        owner_query_df = pd.concat([owner_query_df, temp_df], ignore_index=True)
        owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{yearOfQuery}.txt', sep='\t', index=False)
    
  else:


     for idx, file in enumerate(os.listdir(data_directory)):
      owner_query_df = pd.DataFrame()
      if f"transArchive_{yearOfQuery}" in file:
      

        owner_gbq_query = f"""
        SELECT
          *,
        SAFE_CAST(Scale AS INT64) AS IntScale
        FROM
          `the_wedge_dataset.{file.split('.')[0]}`
        WHERE card_no = 48289.0 OR card_no = 48420.0 OR card_no = 56191.0 OR card_no = 20300.0 OR card_no = 48996.0 OR card_no = 56191.0 OR card_no = 16551.0 
        """

        temp_df = run_query2(owner_gbq_query)

        # #temp_df = pl.DataFrame(results)

        # rows = [dict(row) for row in results]
        # #columns = list(rows[0].keys()) if rows else []

        # # Create Polars DataFrame from the rows
        # temp_df = pl.DataFrame(rows)

        owner_query_df = pd.concat([owner_query_df, temp_df], ignore_index=True)
        owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{file.split('.')[0]}_version2.txt', sep='\t', index=False)


# if results is None or len(results) == 0:
#     print("No results found or query returned None.")
# else:
#     # Process the rows if results exist
#     rows = [dict(row) for row in results]
#     columns = list(rows[0].keys()) if rows else []

#     # Create Polars DataFrame from the rows
#     gbq_query_df = pl.DataFrame(rows)


<>:30: SyntaxWarning: invalid escape sequence '\C'
<>:60: SyntaxWarning: invalid escape sequence '\C'
<>:30: SyntaxWarning: invalid escape sequence '\C'
<>:60: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\4247815683.py:30: SyntaxWarning: invalid escape sequence '\C'
  owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{yearOfQuery}.txt', sep='\t', index=False)
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\4247815683.py:60: SyntaxWarning: invalid escape sequence '\C'
  owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{file.split('.')[0]}_version2.txt', sep='\t', index=False)


In [4]:
for x in range(2010, 2018):
    try:
        save_owner_query_to_file(x, tableByTable=False)
    except Exception as e:
        print(f"Error: {e} with year {x}")
        continue


Error: name 'os' is not defined with year 2010
Error: name 'os' is not defined with year 2011
Error: name 'os' is not defined with year 2012
Error: name 'os' is not defined with year 2013
Error: name 'os' is not defined with year 2014
Error: name 'os' is not defined with year 2015
Error: name 'os' is not defined with year 2016
Error: name 'os' is not defined with year 2017


## Manual Queries for 2015 and 2016

In [24]:
owner_query_df = pd.DataFrame()

fileNameForQuery = "transArchive_201510"

owner_gbq_query = f"""
        SELECT
          *
        FROM
          `the_wedge_dataset.{fileNameForQuery}*`
        WHERE card_no = 48289.0 OR card_no = 48420.0 OR card_no = 56191.0 OR card_no = 20300.0 OR card_no = 48996.0 OR card_no = 56191.0 OR card_no = 16551.0 
        """

temp_df = run_query2(owner_gbq_query)

# #temp_df = pl.DataFrame(results)

# rows = [dict(row) for row in results]
# #columns = list(rows[0].keys()) if rows else []

# # Create Polars DataFrame from the rows
# temp_df = pl.DataFrame(rows)

owner_query_df = pd.concat([owner_query_df, temp_df], ignore_index=True)
owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{fileNameForQuery}.txt', sep='\t', index=False)

<>:24: SyntaxWarning: invalid escape sequence '\C'
<>:24: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\666650559.py:24: SyntaxWarning: invalid escape sequence '\C'
  owner_query_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\owner_data_{fileNameForQuery}.txt', sep='\t', index=False)
c:\Users\justi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data processed: 322.89 MB
Estimated bytes processed: 338571368
Estimated cost: $0.0015396443


Estimated bytes processed against a full year of data: 16928568400
Estimated cost against a full year of data: $0.07698221634200308472
Estimated cost against a full year of data every 6 hours: $112.39403585932450369000

---------------


## Containing the Data into a Single Dataframe and File

In [25]:
columns = ['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id', 'IntScale']

# Create an empty Polars DataFrame with the specified columns
final_owner_df = pd.DataFrame({col: [] for col in columns})



for idx, file in enumerate(os.listdir('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder')):

    try:
        temp_df = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\{file}', sep='\t')
        
        final_owner_df = pd.concat([final_owner_df, temp_df], axis=0)
    except Exception as e:
        print(f"Error: {e} with file {file}")
        continue




<string>:11: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\{'
<>:11: SyntaxWarning: invalid escape sequence '\C'
<string>:11: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\{'
<>:11: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\2019297716.py:8: SyntaxWarning: invalid escape sequence '\C'
  for idx, file in enumerate(os.listdir('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder')):
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\2019297716.py:11: SyntaxWarning: invalid escape sequence '\{'
  temp_df = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\{file}', sep='\t')
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\2019297716.py:11: SyntaxWarning: invalid escape sequence '\C'
  temp_df = pd.read_csv(f'E:\Colle

Error: [Errno 13] Permission denied: 'E:\\College\\Fall 2024\\ADA\\Wedge\\Wedge_Project\\owner_data_folder\\graveyard' with file graveyard


C:\Users\justi\AppData\Local\Temp\ipykernel_2364\2019297716.py:11: DtypeWarning: Columns (33,34,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\{file}', sep='\t')


In [26]:
final_owner_df.drop('IntScale', axis=1, inplace=True)

In [27]:
final_owner_df.to_csv('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data_2.txt', sep='\t', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_2364\3082844610.py:1: SyntaxWarning: invalid escape sequence '\C'
  final_owner_df.to_csv('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data_2.txt', sep='\t', index=False)


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_11028\380248305.py:1: SyntaxWarning: invalid escape sequence '\C'
  wedge_task_two = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data.txt', sep='\t')
C:\Users\justi\AppData\Local\Temp\ipykernel_11028\380248305.py:1: DtypeWarning: Columns (33,34,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  wedge_task_two = pd.read_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\owner_data_folder\_final_owner_data.txt', sep='\t')
